In [ ]:
#modules
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
import math
from collections import OrderedDict
import re
from PIL import Image


import torch
import torch.nn as nn
from torch.nn import init
from torchvision import models, transforms, datasets
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F


In [ ]:
#hyper-parameter
BATCH_size = 64
IMG_size = 256
LR=1e-4 
WEIGHT_decay=5e-4
MODEL_name = 'RESNET_50_SAM_SCHEDULER_AUG' #to stroe the hyperparameter
JITTER_strength = 0.2 #(0.8 to 1.2) 합리적임. 사람 옷들도 다르니까 ㅇㅇ
CROP_size = 224

In [ ]:
#Defininng loader
def one_image_loader(path):
    return Image.open(path).convert('RGB')

class Loader(torch.utils.data.Dataset):
    def __init__(self, rootdir, split_type, ids=None, transform=None):
        self.impath = rootdir
        self.transform = transform
        self.loader = one_image_loader
            

        imnames = []
        imclasses = []
        
        for i in range(10):
            if(split_type == 'train'):
                for j in train_ids[i]:
                    imclasses.append(i)
                    imnames.append(j)
            else :
                for j in val_ids[i]:
                    imclasses.append(i)
                    imnames.append(j)

        self.imnames = imnames
        self.imclasses = imclasses
    
    def __getitem__(self, index):
        original_img = self.loader(os.path.join(self.impath, self.imnames[index]))
        img = self.transform(original_img)
        label = self.imclasses[index]
        return img, label
        
    def __len__(self):
        return len(self.imnames)

In [ ]:
#Test loader
class TestLoader(torch.utils.data.Dataset):
    def __init__(self, rootdir, transform=None):
        self.impath = rootdir
        self.transform = transform
        self.loader = one_image_loader
        self.imnames = os.listdir(rootdir)

    
    def __getitem__(self, index):
        original_img = self.loader(os.path.join(self.impath, self.imnames[index]))
        img = self.transform(original_img)

        return self.imnames[index],img
        
    def __len__(self):
        return len(self.imnames)
    
#loading the test data
test_path = "../input/state-farm-distracted-driver-detection/imgs/test"
test_loader = torch.utils.data.DataLoader(
             TestLoader(test_path,
                    transform=transforms.Compose([
                                   transforms.Resize((IMG_size,IMG_size)),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])),
                               batch_size=BATCH_size, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)
print('test_loader done')
print('loaded {} test images'.format(len(test_loader.dataset)))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
#implement 3 models, RESNET 18, RESNET 50, DENSNET121
__all__ = ['resnet18', 'resnet50', 'densenet121']

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'densenet121': 'https://download.pytorch.org/models/densenet121-a639ec97.pth',
}

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


In [ ]:
#weight initialization functions
def weights_init_kaiming(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
        if m.bias is not None:
            init.constant_(m.bias.data, 0.0)        
    elif classname.find('Linear') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_out')
        if m.bias is not None:
            init.constant_(m.bias.data, 0.0)
    elif classname.find('BatchNorm1d') != -1:
        init.normal_(m.weight.data, 1.0, 0.02)
        if m.bias is not None:        
            init.constant_(m.bias.data, 0.0)
    elif classname.find('BatchNorm2d') != -1:
        init.normal_(m.weight.data, 1.0, 0.02)
        if m.bias is not None:        
            init.constant_(m.bias.data, 0.0)
    elif classname.find('BatchNorm') != -1:
        if m.affine is not None:
            init.constant_(m.weight.data, 1.0)
            init.constant_(m.bias.data, 0.0)        

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('Linear') != -1:
        init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        init.normal_(m.weight.data, 1.0, 0.02)
        init.constant_(m.bias.data, 0.0)

def weights_init_xavier(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.xavier_normal_(m.weight.data, gain=1)
    elif classname.find('Linear') != -1:
        init.xavier_normal_(m.weight.data, gain=1)
    elif classname.find('BatchNorm') != -1:
        init.normal_(m.weight.data, 1.0, 0.02)
        init.constant_(m.bias.data, 0.0)

def weights_init_classifier(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        init.normal_(m.weight.data, std=0.001)
        init.constant_(m.bias.data, 0.0)     


In [ ]:
#blocks implementation
class ClassBlock(nn.Module):
    def __init__(self, input_dim, class_num, dropout=True, relu=True, num_bottleneck=512): 
        super(ClassBlock, self).__init__()
        add_block = []
        add_block += [nn.Linear(input_dim, num_bottleneck)] 
        add_block += [nn.BatchNorm1d(num_bottleneck)]
        if relu: 
            add_block += [nn.ReLU()]
        if dropout: 
            add_block += [nn.Dropout(p=0.5)] 
        add_block = nn.Sequential(*add_block)
        add_block.apply(weights_init_kaiming)
        classifier = []
        classifier += [nn.Linear(num_bottleneck, class_num)]
        classifier = nn.Sequential(*classifier)
        classifier.apply(weights_init_classifier)
        self.add_block = add_block
        self.classifier = classifier
    def forward(self, x):
        x = self.add_block(x)
        x = self.classifier(x)
        return x    

In [ ]:
#models
class Res18_basic(nn.Module):
    def __init__(self, class_num):
        super(Res18_basic, self).__init__()
        fea_dim = IMG_size
        model_ft = models.resnet18(pretrained=False)
        model_ft.avgpool = nn.AdaptiveAvgPool2d((1,1))
        model_ft.fc = nn.Sequential()
        self.model = model_ft
        self.fc_embed = nn.Linear(512, fea_dim)
        self.fc_embed.apply(weights_init_classifier)
        self.classifier = ClassBlock(512, class_num)
        self.classifier.apply(weights_init_classifier)
        
    def forward(self, x):
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)
        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)
        x = self.model.layer4(x)
        x = self.model.avgpool(x)
        fea =  x.view(x.size(0), -1)
        #embed_fea = self.fc_embed(fea)
        pred = self.classifier(fea)
        #return embed_fea, pred 
        return pred

# Define the ResNet18-based Model
class Res18(nn.Module):
    def __init__(self, class_num):
        super(Res18, self).__init__()
        fea_dim = IMG_size
        model_ft = models.resnet18(pretrained=False)
        model_ft.avgpool = nn.AdaptiveAvgPool2d((1,1))
        model_ft.fc = nn.Sequential()
        self.model = model_ft
        self.fc_embed = nn.Linear(512, fea_dim)
        self.fc_embed.apply(weights_init_classifier)
        self.classifier = ClassBlock(512, class_num)
        self.classifier.apply(weights_init_classifier)
        
    def forward(self, x):
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)
        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)
        x = self.model.layer4(x)
        x = self.model.avgpool(x)
        fea =  x.view(x.size(0), -1)
        #embed_fea = self.fc_embed(fea)
        pred = self.classifier(fea)
        #return embed_fea, pred 
        return pred

        
class Res50(nn.Module):
    def __init__(self, class_num):
        super(Res50, self).__init__()
        fea_dim = IMG_size       
        model_ft = models.resnet50(pretrained=False)
        model_ft.avgpool = nn.AdaptiveAvgPool2d((1,1))
        model_ft.fc = nn.Sequential()        
        self.model = model_ft
        self.fc_embed = nn.Linear(2048, fea_dim)
        self.fc_embed.apply(weights_init_classifier)
        self.classifier = ClassBlock(2048, class_num)
        self.classifier.apply(weights_init_classifier)        
        
    def forward(self, x):
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)
        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)
        x = self.model.layer4(x)
        x = self.model.avgpool(x)
        fea =  x.view(x.size(0), -1)
        #embed_fea = self.fc_embed(fea)
        pred = self.classifier(fea)
        #return embed_fea, pred    
        return pred
        
class Dense121(nn.Module):
    def __init__(self, class_num):
        super(Dense121, self).__init__()
        fea_dim = IMG_size      
        model_ft = models.densenet121(pretrained=False)
        model_ft.features.classifier = nn.Sequential()
        model_ft.features.avgpool = nn.AdaptiveAvgPool2d((1,1))
        model_ft.features.fc_embed = nn.Linear(1024, fea_dim)
        model_ft.features.fc_embed.apply(weights_init_classifier)  
        model_ft.classifier = ClassBlock(2048, class_num)
        model_ft.classifier.apply(weights_init_classifier)  
        self.model = model_ft
        
    def forward(self, x):
        x = self.model.features.conv0(x)
        x = self.model.features.norm0(x)
        x = self.model.features.relu0(x)
        x = self.model.features.pool0(x)
        x = self.model.features.denseblock1(x)
        x = self.model.features.transition1(x)
        x = self.model.features.denseblock2(x)
        x = self.model.features.transition2(x)
        x = self.model.features.denseblock3(x)
        x = self.model.features.transition3(x)
        x = self.model.features.denseblock4(x)
        x = self.model.features.norm5(x)
        x = self.model.features.avgpool(x)
        fea =  x.view(x.size(0), -1)
        #embed_fea = self.model.features.fc_embed(fea)
        pred = self.model.classifier(fea)        
        #return embed_fea, pred  
        return pred
        
class Res101(nn.Module):
    def __init__(self, class_num):
        super(Res101, self).__init__()
        fea_dim = IMG_size     
        model_ft = models.resnet101(pretrained=False)
        model_ft.avgpool = nn.AdaptiveAvgPool2d((1,1))
        model_ft.fc = nn.Sequential()        
        self.model = model_ft
        self.fc_embed = nn.Linear(2048, fea_dim)
        self.fc_embed.apply(weights_init_classifier)
        self.classifier = ClassBlock(2048, class_num)
        self.classifier.apply(weights_init_classifier)        
        
    def forward(self, x):
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)
        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)
        x = self.model.layer4(x)
        x = self.model.avgpool(x)
        fea =  x.view(x.size(0), -1)
        #embed_fea = self.fc_embed(fea)
        pred = self.classifier(fea)
        #return embed_fea, pred    
        return pred

In [ ]:
import torch.nn as nn
from torch.hub import load_state_dict_from_url
from torchvision.models import ResNet

from torch import nn


class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class SEBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None,
                 *, reduction=16):
        super(SEBasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes, 1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.se = SELayer(planes, reduction)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.se(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class SEBottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None,
                 *, reduction=16):
        super(SEBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se = SELayer(planes * 4, reduction)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
        out = self.se(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


def se_resnet18(num_classes=1_000):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(SEBasicBlock, [2, 2, 2, 2], num_classes=num_classes)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    return model


def se_resnet34(num_classes=1_000):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(SEBasicBlock, [3, 4, 6, 3], num_classes=num_classes)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    return model


def se_resnet50(num_classes=1_000, pretrained=False):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(SEBottleneck, [3, 4, 6, 3], num_classes=num_classes)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    if pretrained:
        model.load_state_dict(load_state_dict_from_url(
            "https://github.com/moskomule/senet.pytorch/releases/download/archive/seresnet50-60a8950a85b2b.pkl"))
    return model


def se_resnet101(num_classes=1_000):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(SEBottleneck, [3, 4, 23, 3], num_classes=num_classes)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    return model


def se_resnet152(num_classes=1_000):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(SEBottleneck, [3, 8, 36, 3], num_classes=num_classes)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    return model


class CifarSEBasicBlock(nn.Module):
    def __init__(self, inplanes, planes, stride=1, reduction=16):
        super(CifarSEBasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.se = SELayer(planes, reduction)
        if inplanes != planes:
            self.downsample = nn.Sequential(nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False),
                                            nn.BatchNorm2d(planes))
        else:
            self.downsample = lambda x: x
        self.stride = stride

    def forward(self, x):
        residual = self.downsample(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.se(out)

        out += residual
        out = self.relu(out)

        return out


class CifarSEResNet(nn.Module):
    def __init__(self, block, n_size, num_classes=10, reduction=16):
        super(CifarSEResNet, self).__init__()
        self.inplane = 16
        self.conv1 = nn.Conv2d(
            3, self.inplane, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplane)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(
            block, 16, blocks=n_size, stride=1, reduction=reduction)
        self.layer2 = self._make_layer(
            block, 32, blocks=n_size, stride=2, reduction=reduction)
        self.layer3 = self._make_layer(
            block, 64, blocks=n_size, stride=2, reduction=reduction)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(64, num_classes)
        self.initialize()

    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride, reduction):
        strides = [stride] + [1] * (blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inplane, planes, stride, reduction))
            self.inplane = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


class CifarSEPreActResNet(CifarSEResNet):
    def __init__(self, block, n_size, num_classes=10, reduction=16):
        super(CifarSEPreActResNet, self).__init__(
            block, n_size, num_classes, reduction)
        self.bn1 = nn.BatchNorm2d(self.inplane)
        self.initialize()

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.bn1(x)
        x = self.relu(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)


def se_resnet20(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = CifarSEResNet(CifarSEBasicBlock, 3, **kwargs)
    return model


def se_resnet32(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = CifarSEResNet(CifarSEBasicBlock, 5, **kwargs)
    return model


def se_resnet56(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = CifarSEResNet(CifarSEBasicBlock, 9, **kwargs)
    return model


def se_preactresnet20(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = CifarSEPreActResNet(CifarSEBasicBlock, 3, **kwargs)
    return model


def se_preactresnet32(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = CifarSEPreActResNet(CifarSEBasicBlock, 5, **kwargs)
    return model


def se_preactresnet56(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = CifarSEPreActResNet(CifarSEBasicBlock, 9, **kwargs)
    return model

In [ ]:
class SAM(torch.optim.Optimizer): # SAM optimizer from https://github.com/davda54/sam
    def __init__(self, params, base_optimizer, rho=0.05, adaptive=False, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, adaptive=adaptive, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = (torch.pow(p, 2) if group["adaptive"] else 1.0) * p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        ((torch.abs(p) if group["adaptive"] else 1.0) * p.grad).norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

In [ ]:
import math
from torch.optim.lr_scheduler import _LRScheduler

class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)

    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [ ]:
model = Res50(10) #num classes : 10, Resnet50
model.eval()

In [ ]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
n_parameters = sum([p.data.nelement() for p in model.parameters()])
print('  + Number of params: {}'.format(n_parameters))


In [ ]:
model.cuda()
criterion = nn.CrossEntropyLoss()
base_optimizer = optim.Adam
optimizer = SAM(model.parameters(), base_optimizer, lr=0, weight_decay=WEIGHT_decay)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=8, T_mult=2, eta_max=LR,  T_up=2, gamma=0.5)

In [ ]:
'''
2021-06-15 15:28:28.381337: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

2021-06-15 15:28:28.481444: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

found 1992 train, 497 validation images for class 0

found 1814 train, 453 validation images for class 1

found 1854 train, 463 validation images for class 2

found 1877 train, 469 validation images for class 3

found 1861 train, 465 validation images for class 4

found 1850 train, 462 validation images for class 5

found 1860 train, 465 validation images for class 6

found 1602 train, 400 validation images for class 7

found 1529 train, 382 validation images for class 8

found 1704 train, 425 validation images for class 9

train_loader done

loaded 17943 train images

validation_loader done

loaded 4481 validation images

test_loader done

loaded 79726 test images

  + Number of params: 25087818

Epoch 1/50

----------

train loss: 2.2997, acc: 10.0318

model achieved the best accuracy (10.0424%) - saving best checkpoint...

validation loss: 2.3025, acc: 10.0424

Epoch 2/50

----------

train loss: 2.2973, acc: 10.7228

model achieved the best accuracy (11.8500%) - saving best checkpoint...

validation loss: 2.2990, acc: 11.8500

Epoch 3/50

----------

train loss: 2.2874, acc: 11.5923

model achieved the best accuracy (12.2517%) - saving best checkpoint...

validation loss: 2.2358, acc: 12.2517

Epoch 4/50

----------

train loss: 1.9627, acc: 28.3119

model achieved the best accuracy (48.5606%) - saving best checkpoint...

validation loss: 1.3652, acc: 48.5606

Epoch 5/50

----------

train loss: 1.3514, acc: 54.3722

model achieved the best accuracy (77.8621%) - saving best checkpoint...

validation loss: 0.7525, acc: 77.8621

Epoch 6/50

----------

train loss: 0.9214, acc: 69.8824

model achieved the best accuracy (84.1776%) - saving best checkpoint...

validation loss: 0.5334, acc: 84.1776

Epoch 7/50

----------

train loss: 0.7593, acc: 75.6340

model achieved the best accuracy (90.9618%) - saving best checkpoint...

validation loss: 0.4483, acc: 90.9618

Epoch 8/50

----------

train loss: 0.6809, acc: 78.5933

model achieved the best accuracy (92.4794%) - saving best checkpoint...

validation loss: 0.3702, acc: 92.4794

Epoch 9/50

----------

train loss: 0.6632, acc: 79.6132

model achieved the best accuracy (92.7695%) - saving best checkpoint...

validation loss: 0.3719, acc: 92.7695

Epoch 10/50

----------

train loss: 0.6586, acc: 79.5129

model achieved the best accuracy (94.8003%) - saving best checkpoint...

validation loss: 0.3137, acc: 94.8003

Epoch 11/50

----------

train loss: 0.6214, acc: 80.2932

validation loss: 0.3307, acc: 92.7025

Epoch 12/50

----------

train loss: 0.5647, acc: 82.4054

model achieved the best accuracy (95.9830%) - saving best checkpoint...

validation loss: 0.2079, acc: 95.9830

Epoch 13/50

----------

train loss: 0.5079, acc: 83.9269

model achieved the best accuracy (96.3624%) - saving best checkpoint...

validation loss: 0.1932, acc: 96.3624

Epoch 14/50

----------

train loss: 0.4563, acc: 85.5765

model achieved the best accuracy (97.4559%) - saving best checkpoint...

validation loss: 0.1484, acc: 97.4559

Epoch 15/50

----------

train loss: 0.4268, acc: 86.2453

validation loss: 0.1412, acc: 97.3890

Epoch 16/50

----------

train loss: 0.3965, acc: 87.4714

model achieved the best accuracy (97.9023%) - saving best checkpoint...

validation loss: 0.1141, acc: 97.9023

Epoch 17/50

----------

train loss: 0.3607, acc: 88.6251

model achieved the best accuracy (98.3932%) - saving best checkpoint...

validation loss: 0.0928, acc: 98.3932

Epoch 18/50

----------

train loss: 0.3541, acc: 88.6920

validation loss: 0.1026, acc: 98.1477

Epoch 19/50

----------

train loss: 0.3352, acc: 89.5001

model achieved the best accuracy (98.7280%) - saving best checkpoint...

validation loss: 0.0836, acc: 98.7280

Epoch 20/50

----------

train loss: 0.3080, acc: 90.5701

model achieved the best accuracy (98.9065%) - saving best checkpoint...

validation loss: 0.0765, acc: 98.9065

Epoch 21/50

----------

train loss: 0.3031, acc: 90.5534

validation loss: 0.0774, acc: 98.8842

Epoch 22/50

----------

train loss: 0.3074, acc: 90.5534

model achieved the best accuracy (98.9511%) - saving best checkpoint...

validation loss: 0.0757, acc: 98.9511

Epoch 23/50

----------

train loss: 0.2997, acc: 90.5757

model achieved the best accuracy (98.9734%) - saving best checkpoint...

validation loss: 0.0753, acc: 98.9734

Epoch 24/50

----------

train loss: 0.3230, acc: 89.7565

validation loss: 0.0721, acc: 98.8172

Epoch 25/50

----------

train loss: 0.3200, acc: 89.9348

validation loss: 0.0834, acc: 98.2370

Epoch 26/50

----------

train loss: 0.3272, acc: 89.4834

validation loss: 0.0884, acc: 98.3932

Epoch 27/50

----------

train loss: 0.3133, acc: 89.8679

validation loss: 0.0793, acc: 98.5494

Epoch 28/50

----------

train loss: 0.3066, acc: 90.3138

model achieved the best accuracy (99.0181%) - saving best checkpoint...

validation loss: 0.0619, acc: 99.0181

Epoch 29/50

----------

train loss: 0.2858, acc: 90.9324

model achieved the best accuracy (99.1297%) - saving best checkpoint...

validation loss: 0.0713, acc: 99.1297

Epoch 30/50

----------

train loss: 0.2928, acc: 90.5869

validation loss: 0.0636, acc: 98.9065

Epoch 31/50

----------

train loss: 0.2843, acc: 90.9491

validation loss: 0.0585, acc: 99.0850

Epoch 32/50

----------

train loss: 0.2762, acc: 91.2111

model achieved the best accuracy (99.1520%) - saving best checkpoint...

validation loss: 0.0586, acc: 99.1520

Epoch 33/50

----------

train loss: 0.2710, acc: 91.4228

validation loss: 0.0524, acc: 99.1073

Epoch 34/50

----------

train loss: 0.2651, acc: 91.7349

validation loss: 0.0547, acc: 99.1520

Epoch 35/50

----------

train loss: 0.2607, acc: 91.8018

model achieved the best accuracy (99.2412%) - saving best checkpoint...

validation loss: 0.0505, acc: 99.2412

Epoch 36/50

----------

train loss: 0.2418, acc: 92.4093

model achieved the best accuracy (99.3528%) - saving best checkpoint...

validation loss: 0.0489, acc: 99.3528

Epoch 37/50

----------

train loss: 0.2470, acc: 92.2087

validation loss: 0.0430, acc: 99.3528

Epoch 38/50

----------

train loss: 0.2329, acc: 92.6545

model achieved the best accuracy (99.4421%) - saving best checkpoint...

validation loss: 0.0417, acc: 99.4421

Epoch 39/50

----------

train loss: 0.2330, acc: 92.7716

validation loss: 0.0407, acc: 99.3305

Epoch 40/50

----------

train loss: 0.2262, acc: 92.9053

validation loss: 0.0426, acc: 99.3975

Epoch 41/50

----------

train loss: 0.2246, acc: 92.8496

validation loss: 0.0434, acc: 99.3975

Epoch 42/50

----------

train loss: 0.2257, acc: 92.9778

validation loss: 0.0384, acc: 99.4198

Epoch 43/50

----------

train loss: 0.2166, acc: 93.3512

validation loss: 0.0381, acc: 99.3082

Epoch 44/50

----------

train loss: 0.2172, acc: 93.3233

validation loss: 0.0392, acc: 99.3751

Epoch 45/50

----------

train loss: 0.2171, acc: 93.2453

model achieved the best accuracy (99.4867%) - saving best checkpoint...

validation loss: 0.0402, acc: 99.4867

Epoch 46/50

----------

train loss: 0.2100, acc: 93.6410

validation loss: 0.0394, acc: 99.3975

Epoch 47/50

----------

train loss: 0.2123, acc: 93.2731

validation loss: 0.0394, acc: 99.3528

Epoch 48/50

----------

train loss: 0.2132, acc: 93.2620

validation loss: 0.0384, acc: 99.4644

Epoch 49/50

----------

train loss: 0.2163, acc: 93.2063

validation loss: 0.0378, acc: 99.4644

Epoch 50/50

----------

train loss: 0.2161, acc: 93.2564

validation loss: 0.0382, acc: 99.3975

checkpoint is loaded !

loaded models best accuracy : 99.4867

testing the loaded model

'''

In [ ]:
#analyze the results
ckpt = torch.load('../input/exp-15-1/exp_15_1')

#1. load the trained model
model.load_state_dict(ckpt['classifier'])
optimizer.load_state_dict(ckpt['optimizer'])
best_acc = ckpt['best_acc']
print('checkpoint is loaded !')
print('loaded model''s best accuracy : %.4f' % best_acc)


In [ ]:
#submission
import pandas as pd

model.eval()
np.set_printoptions(precision=10, suppress=True)

# (1) eval & write the test set's result
print('Writing start..')

f = open('/kaggle/working/submission.csv', 'w')
f.write('img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9\n')

with torch.no_grad():
    for i, data in enumerate(test_loader):
        if not i%100 :
            print(i*100.0/len(test_loader), "% done.")
        name, img_data = data
    
        img_data = img_data.cuda()
        img_data = model(img_data)

        img_data = F.softmax(img_data, dim=1)
        img_data = img_data.detach().cpu().numpy()

        for j in range(img_data.shape[0]):
            f.write(name[j])
            for k in img_data[j]:
                f.write(',')
                f.write(str(k))
            f.write('\n')
f.close()